In [1]:
import math
import os
import random
import utils
import config

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# For GCN-LSTM model
import tensorflow as tf
from tensorflow import keras
from keras import Model, initializers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
import stellargraph as sg
from stellargraph.layer import GCN_LSTM

# For hyperopt (parameter optimization)
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
from hyperopt.pyll.base import scope  # quniform returns float, some parameters require int; use this to force int

# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

%matplotlib inline

In [2]:
seed_value = 42
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
def format_scale_data(data, seq_len=1):
    X_total = data.values

    X_total = X_total[(53-seq_len):, :]
    scaler, X_total_scaled = utils.scale(X_total)
    data = X_total_scaled.T
    
    X, y = utils.sequence_data_preparation(seq_len, 1, data)
    y = y[:, 1].reshape((-1, 1))
    features = 13
    obs = seq_len*features

    trainX = X[:140, :, :]
    trainY = y[:140, :]
    validX1 = X[140:144, :, :]
    validY1 = y[140:144, :]
    validX2 = X[144:148, :, :]
    validY2 = y[144:148, :]
    testX = X[148:, :, :]
    testY = y[148:, :]
    
    print(trainX.shape, validX1.shape, validX2.shape, testX.shape)
    print(trainY.shape, validY1.shape, validY2.shape, testY.shape)
    
    y_scaler, y2 = utils.scale(X_total[:, 1].reshape((len(X_total), 1)))
    y2 = X_total[:, -features:]
    print(y2.shape)
    
    return trainX, trainY, validX1, validY1, validX2, validY2, testX, testY, scaler, y_scaler

In [4]:
def run():
    def f_nn(params):
        # Generate data with given window
        saltlake_week = pd.read_csv('../saltlake_week.csv')
        data = saltlake_week[['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate', 'PRCP', 'SNWD',
                              'Percent_Fully_Vaccinated_5&Older', 'TAVG',
                              'Stay at Home', 'Mask', 'School Opening', 'Health Emergency', 'Holiday']]
        trainX, trainY, validX1, validY1, validX2, validY2, testX, testY, scaler, y_scaler = format_scale_data(data=data, 
                                                                                                               seq_len=params['weeks'])
        
        gc_layers = [params['gcn_units_1'], params['gcn_units_2']]
        gc_act = ['relu' for i in range(2)]
        lstm_layers = [params['lstm_units_1'], params['lstm_units_2'], params['lstm_units_3'], params['lstm_units_4'],
                       params['lstm_units_5'], params['lstm_units_6']]
        lstm_act = ['relu' for i in range(6)]
        
        # GCN-LSTM model
        model = GCN_LSTM(
                seq_len=params['weeks'],
                adj=config.TOTAL_ADJACENT_MATRIX,
                gc_layer_sizes=gc_layers,
                gc_activations=gc_act,
                lstm_layer_sizes=lstm_layers,
                lstm_activations=lstm_act,
                dropout = params['dropout'],
                kernel_initializer=initializers.Identity(gain=1)
        )

        x_input, x_output = model.in_out_tensors()
        model = Model(inputs=x_input, outputs=x_output)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=params['learning_rate']), loss="mse",
              metrics=["mse"])

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

        result = model.fit(trainX, trainY, verbose=0, validation_data=(validX1, validY1),
                            batch_size=4,
                            epochs=50,
                            callbacks=[TqdmCallback(verbose=1)]
                            )

        # get the lowest validation loss of the training epochs
        validation_loss = np.amin(result.history['val_loss'])
        print('Best validation loss of epoch:', validation_loss)


        return {'loss': validation_loss, 'status': STATUS_OK, 'model': model, 'params': params}

    # hyperparameters to search over with hyperopt
    space = {'gcn_units_1': scope.int(hp.quniform('gcn_units_1', 4, 32, 2)),
             'gcn_units_2': scope.int(hp.quniform('gcn_units_2', 4, 32, 2)),
             'lstm_units_1': scope.int(hp.quniform('lstm_units_1', 8, 128, 4)),
             'lstm_units_2': scope.int(hp.quniform('lstm_units_2', 8, 128, 4)),
             'lstm_units_3': scope.int(hp.quniform('lstm_units_3', 8, 128, 4)),
             'lstm_units_4': scope.int(hp.quniform('lstm_units_4', 8, 128, 4)),
             'lstm_units_5': scope.int(hp.quniform('lstm_units_5', 8, 128, 4)),
             'lstm_units_6': scope.int(hp.quniform('lstm_units_6', 8, 128, 4)),
             'weeks': scope.int(hp.quniform('weeks', 1, 10, 1)), 
             'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
             'dropout': hp.uniform('dropout', 0.01, 0.5)
             }

    trials = Trials()
    best = fmin(f_nn, space, algo=tpe.suggest, max_evals=20, trials=trials)

    # get best model
    best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
    best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
    
    # save best model
    print(best_params)
    print(best_model.summary())
    best_model.save('Model/GCN-LSTM-2_6_new')
    
    # load data
    saltlake_week = pd.read_csv('../saltlake_week.csv')
    data = saltlake_week[['Cases', 'VMT (Veh-Miles)', 'News Sentiment', 'Unemployment Rate', 'PRCP', 'SNWD',
                          'Percent_Fully_Vaccinated_5&Older', 'TAVG', 
                          'Stay at Home', 'Mask', 'School Opening', 'Health Emergency', 'Holiday']]
    trainX, trainY, validX1, validY1, validX2, validY2, testX, testY, scaler, y_scaler = format_scale_data(data=data, 
                                                                                                        seq_len=best_params['weeks'])
    
    # evaluate model on second validation set
    best_model.evaluate(validX2, validY2)
    
    # invert predictions of model
    yhat_valid2 = best_model.predict(validX2)[:, 1].reshape((-1, 1))
    yhat_valid2_inv = y_scaler.inverse_transform(yhat_valid2)
    validY2_inv = y_scaler.inverse_transform(validY2.reshape((-1, 1)))
    
    print(yhat_valid2.shape)
    print(yhat_valid2_inv.shape)
    print(validY2_inv.shape)
    
    yhat_train = best_model.predict(trainX)[:, 1].reshape((-1, 1))
    yhat_train_inv = y_scaler.inverse_transform(yhat_train)
    trainY_inv = y_scaler.inverse_transform(trainY.reshape((-1, 1)))
    
    # evaluate model with inverted features
    print("Mean Squared Error: {}".format(mean_squared_error(validY2_inv, yhat_valid2_inv)))
    print("Root Mean Squared Error: {}".format(mean_squared_error(validY2_inv, yhat_valid2_inv, squared=False)))
    print("Mean Absolute Percentage Error: {}".format(mean_absolute_percentage_error(validY2_inv, yhat_valid2_inv)))
    
    # plot model predictions
    plt.figure()
    plt.plot(yhat_valid2_inv, label='Predicted')
    plt.plot(validY2_inv, label='True')
    plt.legend()
    plt.title("Validation Data 2")
    plt.show()
    
    plt.figure()
    plt.plot(yhat_train_inv, label='Predicted')
    plt.plot(trainY_inv, label='True')
    plt.legend()
    plt.title("Training Data")
    plt.show()
    
    return best_model

In [ ]:
gcn_lstm_2_6_new = run()